# Kaggle 신용카드 사기 검출 
https://www.kaggle.com/mlg-ulb/creditcardfraud
## Credit Card Fraud Detection
* creditcard.csv (284,807 * 31)
* Class : <font color = 'blue'>'0' (정상결제)</font>, <font color = 'red'>'1' (부정결제)</font>
* 사기 검출(Fraud Detection), 이상 탐지(Anomaly Detection)

In [2]:
import warnings
warnings.filterwarnings('ignore')

# I. wget From Github
* 'creditCardFraud.zip' 파일 다운로드

In [ ]:
# !wget https://raw.githubusercontent.com/rusita-ai/pyData/master/creditCardFraud.zip

--2023-01-12 01:34:27--  https://raw.githubusercontent.com/rusita-ai/pyData/master/creditCardFraud.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155672 (66M) [application/zip]
Saving to: ‘creditCardFraud.zip’

creditCardFraud.zip 100%[===================>]  65.95M   179MB/s    in 0.4s    

2023-01-12 01:34:32 (179 MB/s) - ‘creditCardFraud.zip’ saved [69155672/69155672]



* 다운로드 결과 확인

In [ ]:
# !ls -l 

total 67540
-rw-r--r-- 1 root root 69155672 Jan 12 01:34 creditCardFraud.zip
drwxr-xr-x 1 root root     4096 Jan 10 14:33 sample_data


# II. Data Preprocessing

> ## 1) Unzip 'creditCardFraud.zip'

* Colab 파일시스템에 'creditcard.csv' 파일 생성

In [ ]:
# !unzip /content/creditCardFraud.zip

Archive:  /content/creditCardFraud.zip
  inflating: creditcard.csv          


* creditcard.csv 파일 확인

In [ ]:
# !ls -l

total 214836
-rw-r--r-- 1 root root 150828752 Sep 20  2019 creditcard.csv
-rw-r--r-- 1 root root  69155672 Jan 12 01:34 creditCardFraud.zip
drwxr-xr-x 1 root root      4096 Jan 10 14:33 sample_data


> ## 2) 데이터 읽어오기

* pandas DataFrame

In [1]:
import pandas as pd

DF = pd.read_csv('../../../secret/homework_data/ML/creditcard.csv')

DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [3]:
DF.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


* 0 (정상) Class와 1 (사기) Class 개수

In [4]:
DF.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

* 0 (정상) Class와 1 (사기) Class 비율

In [5]:
(DF.Class.value_counts() / DF.shape[0]) * 100

0    99.827251
1     0.172749
Name: Class, dtype: float64

> ## 3) 'Time' -> 'hours'

* 'Time': 각 거래와 첫 번째 거래 사이에 경과된 초('Seconds') 

> ### (1) 시간('hours') 정보 생성

In [6]:
timedelta = pd.to_timedelta(DF['Time'], unit = 's')

DF['Time'] = (timedelta.dt.components.hours).astype(int)

In [7]:
DF.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


> ## 4) train_test_split( )

* X (Input), y (Output) 지정

In [8]:
X = DF.iloc[:,:-1]
y = DF.iloc[:, -1]

X.shape, y.shape

((284807, 30), (284807,))

* With 'Stratify'

In [9]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 2045,
                                                    stratify = y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((199364, 30), (199364,), (85443, 30), (85443,))

* Train_Data와 Test_Data의 1 (부정) 비율이 균형

In [10]:
print('Train_Data :','\n', (y_train.value_counts() / y_train.shape[0]) * 100)
print('Test_Data :','\n', (y_test.value_counts() / y_test.shape[0]) * 100)

Train_Data : 
 0    99.827451
1     0.172549
Name: Class, dtype: float64
Test_Data : 
 0    99.826785
1     0.173215
Name: Class, dtype: float64


# III. MLP Modeling

In [23]:
from sklearn.neural_network import MLPClassifier

Model_NN = MLPClassifier(hidden_layer_sizes = (20), # node 5개
                         activation = 'logistic', # sigmoid
                         solver ='adam',  # optimizer
                         max_iter = 5000,
                         random_state = 2045,
                        verbose=3)

Model_NN.fit(X_train, y_train)

Iteration 1, loss = 0.04823566
Iteration 2, loss = 0.00677542
Iteration 3, loss = 0.00501465
Iteration 4, loss = 0.00425132
Iteration 5, loss = 0.00385040
Iteration 6, loss = 0.00359909
Iteration 7, loss = 0.00345853
Iteration 8, loss = 0.00342595
Iteration 9, loss = 0.00327240
Iteration 10, loss = 0.00320549
Iteration 11, loss = 0.00309724
Iteration 12, loss = 0.00315048
Iteration 13, loss = 0.00309488
Iteration 14, loss = 0.00305116
Iteration 15, loss = 0.00301626
Iteration 16, loss = 0.00297544
Iteration 17, loss = 0.00296304
Iteration 18, loss = 0.00290589
Iteration 19, loss = 0.00287762
Iteration 20, loss = 0.00288626
Iteration 21, loss = 0.00284767
Iteration 22, loss = 0.00284193
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='logistic', hidden_layer_sizes=20, max_iter=5000,
              random_state=2045, verbose=3)

In [24]:
y_hat = Model_NN.predict(X_test)

y_hat

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_hat)

array([[85272,    23],
       [   25,   123]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score, recall_score

print('%.8f' % recall_score(y_test, y_hat))

0.83108108


In [27]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_hat, 
                            target_names = ['0정상', '1부정'],
                            digits = 5))

              precision    recall  f1-score   support

         0정상    0.99971   0.99973   0.99972     85295
         1부정    0.84247   0.83108   0.83673       148

    accuracy                        0.99944     85443
   macro avg    0.92109   0.91541   0.91823     85443
weighted avg    0.99943   0.99944   0.99944     85443



# 
# 
# 
# The End
# 
# 
# 